In [21]:
import pandas as pd
import tqdm
import numpy as np

In [67]:
db = pd.read_parquet(
    "/home/sergey/projects/Credit_scoring/raw/train_data/train_data_0.pq", 
    engine="pyarrow"
)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,...,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,...,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,...,3,3,3,4,1,2,4,1,0,0


In [69]:
db[:20]

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,...,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,...,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,...,3,3,3,4,1,2,4,1,0,0


In [4]:
db.describe().iloc[1:, 1:]

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
mean,125946.473431,6.537471,9.271325,8.330264,8.299071,8.309444,6.991781,8.337011,9.740888,2.399848,...,2.313242,2.343014,2.376717,3.576724,1.027562,2.694898,3.353936,1.002386,0.172138,0.223934
std,72269.808269,5.121167,5.773757,4.593511,5.245598,4.464844,5.184165,4.233992,5.799625,1.376391,...,1.247674,1.228473,1.205555,1.035586,0.276702,0.489136,1.057304,0.059093,0.377501,0.416878
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,63441.000000,3.000000,4.000000,5.000000,4.000000,6.000000,1.000000,5.000000,5.000000,2.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,2.000000,3.000000,1.000000,0.000000,0.000000
50%,126375.000000,5.000000,9.000000,9.000000,8.000000,8.000000,6.000000,10.000000,10.000000,2.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,3.000000,4.000000,1.000000,0.000000,0.000000
75%,188997.000000,9.000000,14.000000,11.000000,13.000000,12.000000,12.000000,11.000000,15.000000,2.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,3.000000,4.000000,1.000000,0.000000,0.000000
max,249999.000000,51.000000,19.000000,17.000000,17.000000,16.000000,16.000000,15.000000,19.000000,6.000000,...,3.000000,3.000000,3.000000,4.000000,6.000000,6.000000,5.000000,3.000000,1.000000,1.000000


In [8]:
round(db.describe().iloc[1, 1] + db.describe().iloc[2, 1] * 3)

22

In [58]:
arr = db.values[0].astype(np.int8)
arr

array([ 0,  1, 18,  9,  2,  3, 16, 10, 11,  3,  3,  0,  2, 11,  6, 16,  5,
        4,  8,  1,  1,  1,  1,  1, 16,  2, 17,  1,  1,  1,  0,  0,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        3,  3,  3,  4,  1,  3,  4,  1,  0,  0], dtype=int8)

In [59]:
obj = arr.tobytes()

In [60]:
obj

b'\x00\x01\x12\t\x02\x03\x10\n\x0b\x03\x03\x00\x02\x0b\x06\x10\x05\x04\x08\x01\x01\x01\x01\x01\x10\x02\x11\x01\x01\x01\x00\x00\x03\x03\x03\x03\x03\x03\x03\x03\x03\x04\x03\x03\x03\x03\x03\x03\x03\x03\x04\x03\x03\x03\x04\x01\x03\x04\x01\x00\x00'

In [63]:
np.frombuffer(obj, dtype='int8')

array([ 0,  1, 18,  9,  2,  3, 16, 10, 11,  3,  3,  0,  2, 11,  6, 16,  5,
        4,  8,  1,  1,  1,  1,  1, 16,  2, 17,  1,  1,  1,  0,  0,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        3,  3,  3,  4,  1,  3,  4,  1,  0,  0], dtype=int8)

In [70]:
arr = dict()
values = db.values

for value in values:
    try:
        arr[value[0]].append = value[1:].astype(np.int8)
    except:
        arr[value[0]] = list()
        arr[value[0]].append(value[1:].astype(np.int8))

In [71]:
arr

{0: [array([10,  7,  9,  2, 10,  8,  8, 16,  4,  2,  0,  2, 11,  6, 16,  5,  4,
          8,  1,  1,  1,  1,  1, 15,  2, 17,  0,  1,  1,  0,  0,  0,  0,  0,
          0,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,
          3,  3,  4,  1,  2,  4,  1,  0,  0], dtype=int8)],
 1: [array([14, 12,  6,  1,  8, 14, 11, 11,  2,  3,  0,  2,  5,  6, 16,  5,  4,
          8,  1,  1,  1,  1,  1, 16,  2, 17,  1,  1,  1,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,
          3,  3,  4,  1,  2,  3,  1,  0,  1], dtype=int8)],
 2: [array([ 3, 12,  9,  4,  8,  1, 11,  1,  1,  4,  0,  2,  5,  6, 16,  5,  4,
          8,  1,  1,  1,  1,  1,  3,  2, 17,  0,  1,  1,  3,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,  3,
          3,  3,  4,  1,  2,  3,  1,  1,  1], dtype=int8)],
 3: [array([15, 12,  9,  2, 10,  8,  8, 17,  1,  2,  0,  2,  1,  6, 16,  5,  4,
          8,  1,  1,  1,  1,  1,  6,  

limit_output extension: Maximum message size of 10000 exceeded with 295895 characters